In [1]:
from urllib import request
from pathlib import Path
import argparse
import random
import torch
import gradio as gr
from gradio import ChatMessage
from PIL import Image, ImageOps
import requests
from io import BytesIO

from peft import AutoPeftModelForCausalLM

from transformers import AutoTokenizer, BitsAndBytesConfig

c:\Users\rnltl\anaconda3\envs\gemma\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class HPChatBot:
    def __init__(self,
                 model_path: str = 'rnltls/harrypotter_lexicon_finetune',
                 device_map: str = 'auto',
                 load_in_4_bit: bool = True,
                 **quant_kwargs) -> None:
        self.model = None
        self.tokenizer = None
        self.image_processor = None
        self.conv = None
        self.conv_img = []
        self.img_tensor = []
        self.roles = None
        self.stop_key = None
        self.is_chat = False
        self.is_waldo = False
        self.load_models(model_path,
                         device_map=device_map,
                         load_in_4_bit=load_in_4_bit,
                         **quant_kwargs)

    def load_models(self, model_path: str,
                    device_map: str,
                    load_in_4_bit: bool,
                    **quant_kwargs) -> None:
        model = AutoPeftModelForCausalLM.from_pretrained(
            "rnltls/harrypotter_lexicon_finetune", # YOUR MODEL YOU USED FOR TRAINING
            load_in_4bit = load_in_4_bit,
        )
        tokenizer = AutoTokenizer.from_pretrained("rnltls/harrypotter_lexicon_finetune")

        self.model = AutoPeftModelForCausalLM.from_pretrained(
            "rnltls/harrypotter_lexicon_finetune", # YOUR MODEL YOU USED FOR TRAINING
            load_in_4bit = load_in_4_bit,
        )
        self.tokenizer = AutoTokenizer.from_pretrained(model_path,
                                                       use_fast=False)

    # def setup_image(self, img_path: str) -> None:
    #     """Load and process the image."""
    #     if img_path.startswith('http') or img_path.startswith('https'):
    #         response = requests.get(img_path)
    #         self.conv_img = Image.open(BytesIO(response.content)).convert('RGB')
    #     else:
    #         self.conv_img = Image.open(img_path).convert('RGB')
    #     self.img_tensor = self.image_processor.preprocess(self.conv_img,
    #                                                       return_tensors='pt'
    #                                                       )['pixel_values'].half().cuda()

    # def set_image(self, img) -> None:
    #     self.conv_img.append(img)
    #     self.img_tensor.append(self.image_processor.preprocess(self.conv_img[-1],
    #                                                       return_tensors='pt'
    #                                                       )['pixel_values'].half().cuda())

    # def replace_image(self, img) -> None:
    #     self.conv_img[-1] = img
    #     self.img_tensor[-1] = self.image_processor.preprocess(self.conv_img[-1],
    #                                                       return_tensors='pt'
    #                                                       )['pixel_values'].half().cuda()
        
    # def generate_answer(self, **kwargs) -> str:
    #     """Generate an answer from the current conversation."""
    #     raw_prompt = self.conv.get_prompt()
        
    #     input_ids = tokenizer_image_token(raw_prompt,
    #                                       self.tokenizer,
    #                                       IMAGE_TOKEN_INDEX,
    #                                       return_tensors='pt').unsqueeze(0).cuda()
    #     stopping = KeywordsStoppingCriteria([self.stop_key],
    #                                         self.tokenizer,
    #                                         input_ids)
    #     with torch.inference_mode():
    #         if self.img_tensor is not []:
    #             output_ids = self.model.generate(input_ids,
    #                                             images=self.img_tensor,
    #                                             stopping_criteria=[stopping],
    #                                             **kwargs)
    #     outputs = self.tokenizer.decode(
    #         output_ids[0, input_ids.shape[1]:]
    #     ).strip()
    #     self.conv.messages[-1][-1] = outputs

    #     return outputs.rsplit('</s>', 1)[0]

    # def get_conv_text(self) -> str:
    #     """Return full conversation text."""
    #     return self.conv.get_prompt()
    
    # def set_conv(self, conv):
    #     self.conv = conv
    
    # def get_conv(self):
    #     return self.conv
    
    # def start_new_chat(self,
    #                    do_sample=True,
    #                    temperature=0.2,
    #                    max_new_tokens=1024,
    #                    use_cache=True,
    #                    **kwargs) -> str:
    #     """Start a new chat with a new image."""
        
    #     self.roles = self.conv.roles
    #     if self.conv.sep_style == SeparatorStyle.TWO:
    #         self.stop_key = self.conv.sep2
    #     else:
    #         self.stop_key = self.conv.sep
            
    #     answer = self.generate_answer(do_sample=do_sample,
    #                                   temperature=temperature,
    #                                   max_new_tokens=max_new_tokens,
    #                                   use_cache=use_cache,
    #                                   **kwargs)
    #     return answer, self.conv
    
    # def continue_chat(self,
    #                   do_sample=True,
    #                   temperature=0.2,
    #                   max_new_tokens=1024,
    #                   use_cache=True,
    #                   **kwargs) -> str:
    #     """Continue the existing chat."""
    #     if self.conv is None:
    #         raise RuntimeError("No existing conversation found. Start a new"
    #                            "conversation using the `start_new_chat` method.")
    #     answer = self.generate_answer(do_sample=do_sample,
    #                                   temperature=temperature,
    #                                   max_new_tokens=max_new_tokens,
    #                                   use_cache=use_cache,
    #                                   **kwargs)
    #     return answer
    
    # def split_image(self, tile_size = 336):
    #     img = self.conv_img[-1]
        
    #     width, height = img.size
    #     # 가로, 세로로 패딩이 필요한지 계산
    #     pad_width = (tile_size - width % tile_size) % tile_size
    #     pad_height = (tile_size - height % tile_size) % tile_size

    #     # 좌우 및 상하 패딩 계산
    #     left_pad = pad_width // 2
    #     right_pad = pad_width - left_pad
    #     top_pad = pad_height // 2
    #     bottom_pad = pad_height - top_pad

    #     # 패딩 추가
    #     img_padded = ImageOps.expand(img, (left_pad, top_pad, right_pad, bottom_pad), fill='white')

    #     # 패딩된 이미지의 크기
    #     padded_width, padded_height = img_padded.size
        
    #     tiles = []
    #     for y in range(0, padded_height, tile_size):
    #         for x in range(0, padded_width, tile_size):
    #             tiles.append(img_padded.crop((x, y, x + tile_size, y + tile_size)))

    #     return tiles
    
    # def find_waldo(self,
    #                   do_sample=True,
    #                   temperature=0.2,
    #                   max_new_tokens=1024,
    #                   use_cache=True,
    #                   **kwargs) -> str:
    #     if self.conv is None:
    #         raise RuntimeError("No existing conversation found. Start a new"
    #                            "conversation using the `start_new_chat` method.")
        
    #     # img를 나눈다.
    #     splitted_imgs = self.split_image()
    #     # 나눈 이미지를 loop를 돌린다.
    #     # 동일한 질의를 진행하며, Yes라고 나온 이미지를 저장한다.
    #     # 직전의 질의를 지우고, Yes라고 나온 이미지에서 Waldo가 무엇을 하고있는지 질의로 변경하고, 모든 이미지에 대해서 수행하여 state에 등록한다.
    #     text, origin_img, image_process_mode = self.conv.messages[-2][-1]

    #     state = self.conv
    #     answer_list = []
    #     text = "Find Waldo. if not visible, dont Answer what Waldo is doing. if visible, Answer what Waldo is doing."
    #     for idx, choosen_img in enumerate(splitted_imgs):
    #         self.conv.messages[-2][-1] = (text, choosen_img, image_process_mode)
    #         self.replace_image(choosen_img)
    #         answer = self.generate_answer(do_sample=do_sample,
    #                                   temperature=temperature,
    #                                   max_new_tokens=max_new_tokens,
    #                                   use_cache=use_cache,
    #                                   **kwargs)
    #         answer_list.append(answer)
    #         self.conv.messages[-1][-1] = None
    #         print(f"Answer Split IMG {idx+1}")

    #     self.conv.messages[-2][-1] = ("Where's Waldo?", origin_img, image_process_mode)
    #     self.conv.messages[-1][-1] = "Picture that Waldo is in it and what he's doing."

    #     for answer, choosen_img in zip(answer_list, splitted_imgs):
    #         state.append_message(state.roles[0], ("", choosen_img, image_process_mode))
    #         state.append_message(state.roles[1], (answer))
    #         self.set_image(choosen_img)
    #         self.conv_img.append(choosen_img)
            
    #     self.conv = state
    #     return answer
    
    # def set_chat_type(self, chat_type):
    #     chat_type = chat_type.lower()
    #     if chat_type == 'chat':
    #         self.is_chat = True
    #     elif chat_type == 'waldo':
    #         self.is_waldo = True

    # def clear(self):
    #     self.conv = None
    #     self.img_tensor = []
    #     self.conv_img = []
    #     self.is_chat = False
    #     self.is_waldo = False

HP_chatbot = HPChatBot(load_in_8bit=True,
                bnb_8bit_compute_dtype=torch.float16,
                bnb_8bit_use_double_quant=True,
                bnb_8bit_quant_type='nf8')

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
c:\Users\rnltl\anaconda3\envs\gemma\lib\site-packages\transformers\quantizers\auto.py:174: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
`low_cpu_mem_usage` was None, now set to True since model is quantized.
`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
S

In [4]:
enable_btn = gr.Button(interactive=True)
disable_btn = gr.Button(interactive=False)
spell_chat, potion_chat, other_chat, house_chat = False, False, False, False

question_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

def show_image(path):
    # Convert To PIL Image
    image = Image.open(path)
    return image

def spell_chatting(chat_history, request: gr.Request):
    global spell_chat, potion_chat, other_chat, house_chat
    chat_history.clear()
    print(f"chatting ip: {request.client.host}")
    spell_chat, potion_chat, other_chat, house_chat = True, False, False, False
    bot_message = "Welcome to Magic Spell Class!\nTell me what you want to achieve, and I’ll suggest the perfect spell for it!\nIf you ask in the format: 'What spell can I use when I ~?', I can give you even better suggestions!"
    txt_box = gr.Textbox(value="What spell can I use when I ", interactive=True)
    prof_IMG = show_image("./IMG/Spell_stand.jpg")
    chat_history.append([None,bot_message])
    return chat_history, prof_IMG, txt_box, enable_btn, enable_btn, enable_btn

def potion_chatting(chat_history, request: gr.Request):
    global spell_chat, potion_chat, other_chat, house_chat
    chat_history.clear()
    print(f"chatting ip: {request.client.host}")
    spell_chat, potion_chat, other_chat, house_chat = False, True, False, False
    bot_message = "Welcome to Potion Class!\nTell me what you want to achieve, and I’ll suggest the perfect potion for it!\nIf you ask in the format: 'What potion can I make when I ~?', I can give you even better suggestions!"
    txt_box = gr.Textbox(value="What potion can I make when I ", interactive=True)
    prof_IMG = show_image("./IMG/Potion_stand.jpg")
    chat_history.append([None,bot_message])
    return chat_history, prof_IMG, txt_box, enable_btn, enable_btn, enable_btn

def other_chatting(chat_history, request: gr.Request):
    global spell_chat, potion_chat, other_chat, house_chat
    chat_history.clear()
    print(f"chatting ip: {request.client.host}")
    spell_chat, potion_chat, other_chat, house_chat = False, False, True, False
    bot_message = "Welcome to Library!\nWelcome to the library! Feel free to ask me anything if you're curious!"
    txt_box = gr.Textbox(placeholder="Ask anything you're curious about in the Wizarding World!", value="", interactive=True)
    prof_IMG = show_image("./IMG/Other_stand.jpg")
    chat_history.append([None,bot_message])
    return chat_history, prof_IMG, txt_box, enable_btn, enable_btn, enable_btn

def house_chatting(chat_history, request: gr.Request):
    global spell_chat, potion_chat, other_chat, house_chat
    chat_history.clear()
    print(f"chatting ip: {request.client.host}")
    spell_chat, potion_chat, other_chat, house_chat = False, False, False, True
    bot_message = "Hmm, let's see… where shall I place you?"
    txt_box = gr.Textbox(placeholder="faiosdjfopasdjfoasdjfopsd", value="", interactive=True)
    prof_IMG = show_image("./IMG/sorting_hat.jpg")
    chat_history.append([None,bot_message])
    return chat_history, prof_IMG, txt_box, enable_btn, enable_btn, enable_btn

def ask_question(chat_history, text_data, request: gr.Request):
    global spell_chat, potion_chat, other_chat, house_chat
    print(spell_chat, potion_chat, other_chat, house_chat)
    if spell_chat:
        print(text_data)
        prof_IMG = show_image("./IMG/Spell_think.jpg")
        chat_history.append([text_data, None])
        text_data = gr.Textbox(value="What spell can I use when I ", interactive=True)
        return chat_history, prof_IMG, text_data
    
    if potion_chat:
        print(text_data)
        prof_IMG = show_image("./IMG/Potion_think.jpg")
        chat_history.append([text_data, None])
        text_data = gr.Textbox(placeholder="Ask anything you're curious about in the Wizarding World!", value="What potion can I make when I ", interactive=True)
        return chat_history, prof_IMG, text_data
    
    if other_chat:
        print(text_data)
        prof_IMG = show_image("./IMG/Other_think.jpg")
        chat_history.append(["test message", None])
        text_data = gr.Textbox(placeholder="faiosdjfopasdjfoasdjfopsd", value="", interactive=True)
        return chat_history, prof_IMG, text_data
    
    if house_chat:
        print(text_data)
        prof_IMG = show_image("./IMG/sorting_hat.jpg")
        chat_history.append(["test message", None])
        text_data = gr.Textbox(value="asdfasdfasdfasdf", interactive=True)
        return chat_history, prof_IMG, text_data

background_image_path = Path("./IMG/bg1.jpg").resolve()



custom_css = f"""

body {{
    background-image: url('file://{background_image_path}');
    background-size: cover;  /* 배경 이미지가 화면 전체를 덮도록 설정 */
    background-repeat: no-repeat;  /* 이미지 반복 방지 */
    background-position: center;  /* 이미지 가운데 정렬 */
}}
"""
custom_css = f"""
body {{
    background-image: url('file://{background_image_path}');
    background-size: cover;  /* Make the image cover the entire background */
    background-position: center;  /* Center the background image */
    background-repeat: no-repeat;  /* Do not repeat the background image */
}}

.dark body {{
    background-image: url('file://{background_image_path}');
    background-size: cover;
    background-position: center;
    background-repeat: no-repeat;
    background-color: #333;  /* Optional: Fallback color for dark mode */
}}

.gradio-container {{
    background: transparent !important;  /* Make container background transparent to see the full background image */
}}
"""
def build_gradio(concurrency_count=10):
    textbox = gr.Textbox(show_label=False, placeholder="Welcome to Hogwart!", container=False, interactive = False)
    with gr.Blocks(
            css=custom_css,
            theme='abidlabs/pakistan',
            
            
        ) as demo:
        state = gr.State()
        with gr.Row():
            with gr.Column(scale=3):

                # imagebox = gr.Image(interactive=False)
                imagebox = gr.Image(value=show_image("./IMG/HogwartGemma.jpg"), type="pil", interactive=False, show_label=False, show_download_button=False, show_fullscreen_button=False)
                # image_process_mode = gr.Radio(
                #     ["Crop", "Resize", "Pad", "Default"],
                #     value="Default",
                #     label="Preprocess for non-square image", visible=False)
                
                spell_btn = gr.Button(value="🪄 Spell", interactive = True, scale = 2)
                potion_btn = gr.Button(value="🧉 Potion", interactive = True, scale = 2)
                other_btn = gr.Button(value="📚 Others", interactive = True, scale = 2)
                house_btn = gr.Button(value="🌟 Find your House", interactive = True, scale = 2)

            with gr.Column(scale=8):
                chatbot = gr.Chatbot(
                    label='Free to ask!',
                    height=600,
                )
                with gr.Row():
                    with gr.Column(scale=8):
                        textbox.render()
                    with gr.Column(scale=1, min_width=50):
                        submit_btn = gr.Button(value="Send", variant="primary", interactive = False)

                with gr.Row(elem_id="buttons") as button_row:
                    finish_btn = gr.Button(value="🏁 End Talking", interactive = False)
                    clear_btn = gr.Button(value="🗑️  Clear", interactive=False)    
        box_list = [spell_btn, potion_btn, other_btn, house_btn, textbox, submit_btn, finish_btn, clear_btn]
        spell_btn.click(
            spell_chatting,
            inputs=[chatbot],
            outputs=[chatbot, imagebox, textbox, submit_btn, finish_btn, clear_btn]
        )
        potion_btn.click(
            potion_chatting,
            inputs=[chatbot],
            outputs=[chatbot, imagebox, textbox, submit_btn, finish_btn, clear_btn]
        )
        other_btn.click(
            other_chatting,
            inputs=[chatbot],
            outputs=[chatbot, imagebox, textbox, submit_btn, finish_btn, clear_btn]
        )
        house_btn.click(
            house_chatting,
            inputs=[chatbot],
            outputs=[chatbot, imagebox, textbox, submit_btn, finish_btn, clear_btn]
        )
        submit_btn.click(
            ask_question,
            inputs = [chatbot, textbox],
            outputs = [chatbot, imagebox, textbox]
        )
        finish_btn.click()
        clear_btn.click()

    return demo

import argparse
if __name__ == "__main__":

    # 컨트롤러에서 사용가능한 모델 가져옴. 나는 모델 하나만 쓸 것이기 때문에, get_model_list()에서 해당 모델이 동작하고 있는 url을 넘겨주면 된다!
    # models = [args.model_url] 
    
    # HP_chatbot = HPChatBot(load_in_8bit=True,
    #                    bnb_8bit_compute_dtype=torch.float16,
    #                    bnb_8bit_use_double_quant=True,
    #                    bnb_8bit_quant_type='nf8')
    
    # Gradio를 이용해서 데모 만들기
    demo = build_gradio()
    demo.queue(
        api_open=False
    ).launch(
        server_port=7860,
        share=True
    )
    


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://848ba3ef089029795e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [3]:
demo.close()

Closing server running on port: 7860
